In [11]:
import sys
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn.pool import radius_graph
from torch_geometric.transforms import FixedPoints
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import Cartesian

sys.path.append('..')
import aegnn
from aegnn.datasets.utils.normalization import normalize_time
from events_utils import load_events

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
model = aegnn.models.networks.GraphWen(dataset='ncars', input_shape=torch.tensor([346,260,1]), num_outputs=2).cuda()
# model = aegnn.asyncronous.make_model_asynchronous(model, r=1)

e:\GitHub\aegnn\notebooks\..\aegnn\models\networks\graph_wen.py:24: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self.pool_final = MaxPoolingX(input_shape[:2] // 2, size=4)


In [3]:
model_mask = aegnn.models.networks.GraphMask(dataset='ncars', input_shape=torch.tensor([346,260,1]), num_outputs=2).cuda()

In [12]:
model_dual = aegnn.models.networks.GraphDual(dataset='ncars', input_shape=torch.tensor([346,260,1]), num_outputs=2).cuda()

e:\GitHub\aegnn\notebooks\..\aegnn\models\networks\graph_mask_test.py:26: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self.pool_final = MaxPoolingX(input_shape[:2] // 2, size=4)


In [4]:
det_model = aegnn.models.DetectionModel('graph_wen', 
                                        dataset='ncars',
                                        num_classes=2,
                                        img_shape=(346,260), 
                                        dim=3).cuda()

In [14]:
file_path = r'C:\Users\miracleyoo\OneDrive - kcchub\exp_dataset\bkg_on\events.h5'
events = load_events(file_path, to_df=False)

In [15]:
n_samples = 10000
sampler = FixedPoints(num=n_samples, allow_duplicates=False, replace=False)

In [16]:
events = events[:,np.r_[1,2,0,3]]
events = torch.from_numpy(events.astype(float)).cuda().float()
x, pos = events[:, -1:], events[:, :3]
graph_data = Data(x=x, pos=pos).to('cuda')
graph_data.pos[:, 2] = normalize_time(graph_data.pos[:, 2])
graph_data = sampler(graph_data)
graph_data.edge_index = radius_graph(graph_data.pos, r=3.0, max_num_neighbors=True)
# 把两个node间距作为edge的attribute，Cartesian意思是笛卡尔坐标的
edge_attr = Cartesian(norm=True, cat=False, max_value=None)
graph_data = edge_attr(graph_data)

In [17]:
data_list = [graph_data,graph_data]
loader = DataLoader(data_list, batch_size=2)
batch = next(iter(loader))

In [8]:
graph_data

Data(x=[10000, 1], pos=[10000, 3], edge_index=[2, 2759], edge_attr=[2759, 3])

In [21]:
batch

DataBatch(x=[20000, 1], pos=[20000, 3], edge_index=[2, 5432], edge_attr=[5432, 3], batch=[20000], ptr=[3])

In [11]:
# graph_data.edge_attr[:100]

In [12]:
batch.edge_attr.shape

torch.Size([5606, 3])

In [13]:
batch.x

tensor([[1.],
        [1.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]], device='cuda:0')

In [14]:
res = model(batch)

torch.Size([176, 2])


In [9]:
res_masks = model_mask(batch)
print(batch.x.size())
print(res_masks.shape)

torch.Size([20000, 1])
torch.Size([20000, 1])


In [18]:
pred_labels, pred_masks = model_dual(batch)

torch.Size([20000, 2])


In [19]:
print(batch.x.size())
print(pred_labels.shape)
print(pred_masks.shape)

torch.Size([20000, 1])
torch.Size([2, 2])
torch.Size([20000, 1])


torch.Size([20000, 64])

In [35]:
res_masks.shape

torch.Size([176, 512])

In [22]:
events[20000000:20000010]

tensor([[309.0000, 140.0000,  26.4502,   0.0000],
        [214.0000,  26.0000,  26.4502,   0.0000],
        [ 54.0000,  27.0000,  26.4502,   1.0000],
        [298.0000, 136.0000,  26.4502,   1.0000],
        [ 50.0000,  27.0000,  26.4502,   1.0000],
        [233.0000,  10.0000,  26.4502,   0.0000],
        [ 47.0000, 151.0000,  26.4502,   0.0000],
        [315.0000, 139.0000,  26.4502,   1.0000],
        [317.0000,  92.0000,  26.4502,   1.0000],
        [231.0000,  47.0000,  26.4502,   0.0000]], device='cuda:0',
       dtype=torch.float64)

In [10]:
import torch
from torch_spline_conv import spline_conv

x = torch.rand((4, 2), dtype=torch.float)  # 4 nodes with 2 features each
edge_index = torch.tensor([[0, 1, 1, 2, 2, 3], [1, 0, 2, 1, 3, 2]])  # 6 edges
pseudo = torch.rand((6, 2), dtype=torch.float)  # two-dimensional edge attributes
weight = torch.rand((25, 2, 4), dtype=torch.float)  # 25 parameters for in_channels x out_channels
kernel_size = torch.tensor([5, 5])  # 5 parameters in each edge dimension
is_open_spline = torch.tensor([1, 1], dtype=torch.uint8)  # only use open B-splines
degree = 1  # B-spline degree of 1
norm = True  # Normalize output by node degree.
root_weight = torch.rand((2, 4), dtype=torch.float)  # separately weight root nodes
bias = None  # do not apply an additional bias

out = spline_conv(x, edge_index, pseudo, weight, kernel_size,
                  is_open_spline, degree, norm, root_weight, bias)

print(out.size())
torch.Size([4, 4])

torch.Size([4, 4])


torch.Size([4, 4])